# ag2gcal - Texas A&M Planting Calendar to gCal

Convert the calendars at `calendar_source_urls` to Google Calendar event entries at `calendar_destination_ids`.

Example:

![](img/ag2gcal.png)

In [1]:
import datetime
from datetime import timedelta
import os
import base64

import pandas as pd
import dateparser
import gcsa
import gcsa.recurrence

from gcsa.event import Event
from gcsa.google_calendar import GoogleCalendar

### Various variables

- `region_columns` - list of all regions contained in the tables
- `all_columns` - includes the leftmost index for the veggie
- `gcloud_creds_path` - OAuth credentials with permissions to access the person's calendars
- `calendar_source_urls` - source sites for the planting schedules
- `calendar_destination_ids` - map of region to gCal calendar ID where we'll eventually write the calendar events

In [2]:
region_columns = ['Region I', 'Region II', 'Region III', 'Region IV', 'Region V']
all_columns = ['Vegetable'] + region_columns

In [3]:
gcloud_creds_path = os.environ['GOOGLE_CLOUD_AG2GCAL_CREDS_PATH']

In [4]:
calendar_source_urls = {
    'Spring': 'https://aggie-horticulture.tamu.edu/archives/parsons/earthkind/ekgarden14.html',
    'Fall': 'https://aggie-horticulture.tamu.edu/archives/parsons/fallgarden/falldirect.html'
}

In [5]:
calendar_destination_ids = {
    'Region III': 'c_4kikah5kfau07c7v9t5fieent0@group.calendar.google.com'
}

### Parse date ranges from tables

The dates in the tables are somewhat inconsistent. Play around with them a bit to get them into a useable, two entry array of `[start_date, end_date]`.

In [6]:
def try_parse_date_range(val, year=datetime.datetime.now().year, default_window_in_days=14, date_format='%b %d %Y'):
    stringified_val = str(val)
    
    start_date = None
    end_date = None
    
    if '-' in stringified_val:
        raw_date_range = val.split('-')
        
        if len(raw_date_range) == 2:
            raw_date_range[:] = [f'{raw_date} {year}' for raw_date in raw_date_range]
            
            start_date = raw_date_range[0]
            end_date = raw_date_range[1]
    elif stringified_val.startswith('After'):
        start_date = f'{stringified_val.replace("After", "")} {year}'
    else:
        start_date = f'{stringified_val} {year}'
        
    if start_date != "nat" and isinstance(start_date, str):
        start_date = dateparser.parse(start_date)
        
        if end_date != None:
            end_date = dateparser.parse(end_date)
        
        if start_date != None and end_date == None:
            end_date = start_date + timedelta(days=default_window_in_days)
    
    return [start_date, end_date]
    

### Parse the HTML tables and coerce them in to DataFrames

In [7]:
def parse_url(season, url):
    print(url)
    
    raw_tables = pd.read_html(url)
    
    calendar_table = raw_tables[1]
    
    if calendar_table.iloc[0][0] == 'Vegetables':
        calendar_table.drop([0], inplace=True)

    calendar_table.columns = all_columns
    calendar_table['season'] = season
        
    return calendar_table

### Do it

In [8]:
raw_calendars = pd.DataFrame()

for calendar_source in calendar_source_urls.items():
    calendar = parse_url(*calendar_source)
    raw_calendars = raw_calendars.append(calendar)
    
raw_calendars.reset_index(inplace=True)

https://aggie-horticulture.tamu.edu/archives/parsons/earthkind/ekgarden14.html
https://aggie-horticulture.tamu.edu/archives/parsons/fallgarden/falldirect.html


In [9]:
for region_column in region_columns:
    raw_calendars[region_column] = raw_calendars[region_column].apply(try_parse_date_range)

In [10]:
unpivoted_calendar = raw_calendars.melt(id_vars=['Vegetable', 'season'], value_vars=region_columns, var_name='region', value_name='date_range')

In [11]:
unpivoted_calendar['start_date'] = unpivoted_calendar.apply(lambda row: row['date_range'][0], axis=1)
unpivoted_calendar['end_date'] = unpivoted_calendar.apply(lambda row: row['date_range'][1], axis=1)
unpivoted_calendar.drop(['date_range'], axis=1, inplace=True)

### Establish gcal clients

This uses a desktop OAuth flow, so each calendar will kick out a URL to authorize. You'll need to replace the `&amp;` in the URL with actual ampersands.

TODO: figure out how to output and not urlencode within a given cell.

In [19]:
calendar_clients = {}

for region, calendar_id in calendar_destination_ids.items():
    calendar_clients[region] = GoogleCalendar(calendar_id, credentials_path=gcloud_creds_path)

In [22]:
all_events = {}

for region, client in calendar_clients.items():
    print(f'clearing out old events for {region}')
    
    for event in client:
        client.delete_event(event)
        print(f'deleted event {event.event_id}')

clearing out old events for Region III
deleted event 9g3gabenflgdoc5rueljaebuic
deleted event nsn5j7jlmt4qnosbn5ll374j2c
deleted event 5s6plnd7e71a4b4nkbk60ju8q8
deleted event 6vl69m0o3624875lvf4slmm93o
deleted event jffhe6b67jim6lg2ptvglsi4f0
deleted event mn3q8lq41sor34e3r82mlrkahc
deleted event g29fsmmlknlq3q662a6k81c50k
deleted event b4tkg6lljeqsmsermlm05hq7js
deleted event 2o7oojucm2hu7sl1f5oaumlnr0
deleted event kalat97ppm0hae9ke1k1ndl76k
deleted event u55nmrjlkl29n7al6jv4dii6ko
deleted event bh6a98btom473omesco6gn624s
deleted event 355c8fmoufrl38gqr32svlo3tk
deleted event 0cpo42dmfl0hfjbf452a8jkkc8


### Write events to calendars

Iterate over each of the regions and write the events for that region to the respective calendar defined in `calendar_source_urls`

In [23]:
for region in region_columns:
    print(f'Adding events for {region}')
    
    if region in calendar_clients:
        calendar_client = calendar_clients[region]
    
        for index, row in unpivoted_calendar[unpivoted_calendar['region'] == region].iterrows():
            if row['start_date'] != None and not pd.isnull(row['start_date']):
                humanized_event_id = f'{row["season"]}/{row["Vegetable"]}'

                event = Event(
                    row['Vegetable'],
                    start=row['start_date'],
                    end=row['end_date']
                )

                create_results = calendar_client.add_event(event)
                
                print(f'created {humanized_event_id} -- {create_results.event_id}')

Adding events for Region I
Adding events for Region II
Adding events for Region III
created Spring/Asparagus -- lm6tsq7q77jonglv46ps9tneog
created Spring/Beans, snap bush -- f8ek4v9dikgomdnmrs6hjlb2o0
created Spring/Beans, snap pole -- q4o9sfd5gcu95khlksr5nl61ug
created Spring/Beans, Lima bush -- f6bsjkh2maj8m36m3fq49ka63k
created Spring/Beans, Lima pole -- micc464i9ae9buimhohgj7o8qo
created Spring/Beets -- e0ugprl1cp5cdsa5ecshiua9t0
created Spring/Broccoli -- 3hacgg9ptlm695v74j2nkcokdk
created Spring/Cabbage -- 7d8iatekd8960pfeu0qcdv8s1g
created Spring/Carrots -- thg11vvcku3jrc7asa0rck4djo
created Spring/Cauliflower (transplant) -- cf491pnnn2hkepnf42ujpc9keg
created Spring/Chard, Swiss -- ec7hnh83qa8nk7uf111pvltej8
created Spring/Collards Mar 1-May 1Feb 15-Apr 10Feb 1-Mar 25Jan 15-Mar 15Jan 1-Mar 15 -- bn289oeaarvo1u9t6f7pf13758
created Spring/Corn, sweet -- vk2d9mv73fdvmis6fgv9va5p6g
created Spring/Cucumber -- gjqd219fjet5es34bgmneumltg
created Spring/Eggplant -- otr99nl43q5nl9vdftrg